In [77]:
#Initially forked from Bojan's kernel here: https://www.kaggle.com/tunguz/bow-meta-text-and-dense-features-lb-0-2242/code
#That kernel was forked from Nick Brook's kernel here: https://www.kaggle.com/nicapotato/bow-meta-text-and-dense-features-lgbm?scriptVersionId=3493400
#Used oof method from Faron's kernel here: https://www.kaggle.com/mmueller/stacking-starter?scriptVersionId=390867
#Used some text cleaning method from Muhammad Alfiansyah's kernel here: https://www.kaggle.com/muhammadalfiansyah/push-the-lgbm-v19
import time
notebookstart= time.time()

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc

%matplotlib inline
# Models Packages
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn import feature_selection
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

# Gradient Boosting
import lightgbm as lgb
from sklearn.linear_model import Ridge
from sklearn.cross_validation import KFold

# Tf-Idf
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import FeatureUnion
from scipy.sparse import hstack, csr_matrix
from nltk.corpus import stopwords 

# Viz
import seaborn as sns
import matplotlib.pyplot as plt
import re
import string

NFOLDS = 5
SEED = 42


In [78]:
class SklearnWrapper(object):
    def __init__(self, clf, seed=0, params=None, seed_bool = True):
        if(seed_bool == True):
            params['random_state'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict(x)
        
def get_oof(clf, x_train, y, x_test):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))

    for i, (train_index, test_index) in enumerate(kf):
        print('\nFold {}'.format(i))
        x_tr = x_train[train_index]
        y_tr = y[train_index]
        x_te = x_train[test_index]

        clf.train(x_tr, y_tr)

        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(x_test)

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)
    
def cleanName(text):
    try:
        textProc = text.lower()
        textProc = " ".join(map(str.strip, re.split('(\d+)',textProc)))
        regex = re.compile(u'[^[:alpha:]]')
        textProc = regex.sub(" ", textProc)
        textProc = " ".join(textProc.split())
        return textProc
    except: 
        return "name error"
    
    
def rmse(y, y0):
    assert len(y) == len(y0)
    return np.sqrt(np.mean(np.power((y - y0), 2)))

'''dtypes = {
        'category_name': 'category',
        'parent_category_name': 'category',
        'region': 'category',
        'item_seq_number': 'uint32',
        'user_type': 'category',
        'image_top_1': 'float32',
        'price':'float32',
        'deal_probability': 'float32'
        }'''
print("\nData Load Stage")
training = pd.read_csv('boites_medicaments_train.csv', encoding='utf-8',sep=';')
training['prix'] = training['prix'].apply(np.log)

testing = pd.read_csv('boites_medicaments_test.csv', encoding='utf-8',sep=';')
test_ids=testing['id'].values
ntrain = training.shape[0]
ntest = testing.shape[0]
y = training.prix.values


kf = KFold(ntrain, n_folds=NFOLDS, shuffle=True, random_state=SEED)


testing.drop("id",axis=1, inplace=True)
#training.drop("prix",axis=1, inplace=True)
print('Train shape: {} Rows, {} Columns'.format(*training.shape))
print('Test shape: {} Rows, {} Columns'.format(*testing.shape))

print("Combine Train and Test")
df = pd.concat([training,testing],axis=0)
traindex=df[:ntrain].index
testdex=df[ntrain:].index
#del training, testing
gc.collect()
print('\nAll Data shape: {} Rows, {} Columns'.format(*df.shape))



Data Load Stage
Train shape: 8564 Rows, 41 Columns
Test shape: 3671 Rows, 40 Columns
Combine Train and Test

All Data shape: 12235 Rows, 41 Columns


In [79]:
df.head()

,agrement col,date amm annee,date declar annee,etat commerc,forme pharma,libelle,libelle_ampoule,libelle_capsule,libelle_comprime,libelle_film,...,nb_stylo,nb_tube,prix,statut,statut admin,substances,titulaires,tx rembours,type proc,voies admin
0,oui,2014,2014,Déclaration de commercialisation,comprimé pelliculé,3 plaquette(s) thermoformée(s) PVC-Aluminium d...,0,0,1,0,...,0,0,1.040277,Présentation active,Autorisation active,DÉSOGESTREL,MAJORELLE LUXEMBOURG SOPARFI (LUXEMBOURG),65%,Procédure décentralisée,orale
1,oui,2009,2013,Déclaration de commercialisation,comprimé à croquer,plaquette(s) thermoformée(s) aluminium de 28 c...,0,0,1,0,...,0,0,2.660260,Présentation active,Autorisation active,"MONTÉLUKAST ACIDE, MONTÉLUKAST SODIQUE",TEVA SANTE,65%,Procédure de reconnaissance mutuelle,orale
2,oui,1996,2000,Déclaration de commercialisation,gélule à libération prolongée,plaquette(s) thermoformée(s) PVC-aluminium de ...,0,0,0,0,...,0,0,1.733424,Présentation active,Autorisation active,CHLORHYDRATE DE DILTIAZEM,MYLAN SAS,65%,Procédure nationale,orale
3,oui,2004,2005,Déclaration de commercialisation,granulés à libération prolongée,30 sachet(s)-dose(s) papier aluminium complexe...,0,0,0,0,...,0,0,3.189241,Présentation active,Autorisation active,"ACIDE VALPROÏQUE, VALPROATE DE SODIUM",SANOFI AVENTIS FRANCE,65%,Procédure nationale,orale
4,non,2010,2015,Déclaration d'arrêt de commercialisation,comprimé pelliculé,plaquette(s) thermoformée(s) PVC-Aluminium de ...,0,0,1,0,...,0,0,4.093344,Présentation active,Autorisation active,LÉTROZOLE,TEVA SANTE,100%,Procédure nationale,orale


In [80]:
def feature_Name(df):
    Feat_name={}
    Feat_name['num']=[]
    Feat_name['cat']=[]
    for c in df.columns: 
        if (df[c].dtype)=='O':
              Feat_name['cat'].append(c)
        else : 
            Feat_name['num'].append(c)
    return Feat_name
fname=feature_Name(df)

In [81]:
fname['num']

['date amm annee',
 'date declar annee',
 'libelle_ampoule',
 'libelle_capsule',
 'libelle_comprime',
 'libelle_film',
 'libelle_flacon',
 'libelle_gelule',
 'libelle_pilulier',
 'libelle_plaquette',
 'libelle_poche',
 'libelle_sachet',
 'libelle_seringue',
 'libelle_stylo',
 'libelle_tube',
 'nb_ampoule',
 'nb_capsule',
 'nb_comprime',
 'nb_film',
 'nb_flacon',
 'nb_gelule',
 'nb_ml',
 'nb_pilulier',
 'nb_plaquette',
 'nb_poche',
 'nb_sachet',
 'nb_seringue',
 'nb_stylo',
 'nb_tube',
 'prix']

In [82]:
df[fname['cat']]

,agrement col,etat commerc,forme pharma,libelle,statut,statut admin,substances,titulaires,tx rembours,type proc,voies admin
0,oui,Déclaration de commercialisation,comprimé pelliculé,3 plaquette(s) thermoformée(s) PVC-Aluminium d...,Présentation active,Autorisation active,DÉSOGESTREL,MAJORELLE LUXEMBOURG SOPARFI (LUXEMBOURG),65%,Procédure décentralisée,orale
1,oui,Déclaration de commercialisation,comprimé à croquer,plaquette(s) thermoformée(s) aluminium de 28 c...,Présentation active,Autorisation active,"MONTÉLUKAST ACIDE, MONTÉLUKAST SODIQUE",TEVA SANTE,65%,Procédure de reconnaissance mutuelle,orale
2,oui,Déclaration de commercialisation,gélule à libération prolongée,plaquette(s) thermoformée(s) PVC-aluminium de ...,Présentation active,Autorisation active,CHLORHYDRATE DE DILTIAZEM,MYLAN SAS,65%,Procédure nationale,orale
3,oui,Déclaration de commercialisation,granulés à libération prolongée,30 sachet(s)-dose(s) papier aluminium complexe...,Présentation active,Autorisation active,"ACIDE VALPROÏQUE, VALPROATE DE SODIUM",SANOFI AVENTIS FRANCE,65%,Procédure nationale,orale
4,non,Déclaration d'arrêt de commercialisation,comprimé pelliculé,plaquette(s) thermoformée(s) PVC-Aluminium de ...,Présentation active,Autorisation active,LÉTROZOLE,TEVA SANTE,100%,Procédure nationale,orale
5,oui,Déclaration de commercialisation,comprimé,plaquette(s) thermoformée(s) PVC-aluminium de ...,Présentation active,Autorisation active,FLUTAMIDE,EG LABO - LABORATOIRES EUROGENERICS,100%,Procédure nationale,orale
6,oui,Déclaration de commercialisation,capsule molle,flacon(s) polyéthylène haute densité (PEHD) de...,Présentation active,Autorisation active,BEXAROTÈNE,EISAI (ROYAUME-UNI),100%,Procédure centralisée,orale
7,oui,Déclaration de commercialisation,comprimé,plaquette(s) thermoformée(s) PVC polyéthylène ...,Présentation active,Autorisation active,GLIMÉPIRIDE,SANDOZ,65%,Procédure de reconnaissance mutuelle,orale
8,oui,Déclaration d'arrêt de commercialisation,comprimé sécable,plaquette(s) thermoformée(s) PVC-Aluminium de ...,Présentation active,Autorisation active,MALÉATE D'ÉNALAPRIL,ARROW GENERIQUES,65%,Procédure nationale,orale
9,non,Déclaration de commercialisation,comprimé pelliculé,plaquette(s) polyamide aluminium PVC de 10 com...,Présentation active,Autorisation active,"CEFPODOXIME PROXÉTIL, CEFPODOXIME",ARROW GENERIQUES,65%,Procédure nationale,orale


In [83]:
categorical = list(set(df[fname['cat']])-set(['libelle','titulaires', 'substances', 'forme pharma']))

In [84]:
# Find frequency of is_attributed for each unique value in column
ATTRIBUTION_CATEGORIES = [        
    # V1 Features #
    ###############
    #['app'],
    
    # V2 Features #
    ###############


    ['substances'],
    ['titulaires'],

    
    
    # V3 Features #
    ###############
 

]
freqs = {}
for cols in ATTRIBUTION_CATEGORIES :
    
    # New feature name
    new_feature = '_'.join(cols)+'_confRate'    
    
    # Perform the groupby
    group_object = df[:ntrain].groupby(cols)
    
    # Group sizes    
    group_sizes = group_object.size()
    log_group = np.log(500) # 1000 views -> 60% confidence, 100 views -> 40% confidence 
    print(">> Calculating confidence-weighted rate for: {}.\n   Saving to: {}. Group Max /Mean / Median / Min: {} / {} / {} / {}".format(
        cols, new_feature, 
        group_sizes.max(), 
        np.round(group_sizes.mean(), 2),
        np.round(group_sizes.median(), 2),
        group_sizes.min()
    ))
    
    # Aggregation function
    def rate_calculation(x):
        """Calculate the attributed rate. Scale by confidence"""
        rate = x.sum() / float(x.count())
        conf = np.min([1, np.log(x.count()) / log_group])
        return rate * conf
    
    # Perform the merge
    df = df.merge(
        group_object['prix']. \
            apply(rate_calculation). \
            reset_index(). \
            rename( 
                index=str,
                columns={'prix': new_feature}
            )[cols + [new_feature]],
        on=cols, how='left'
    )
    
df.head()
df.drop("prix",axis=1, inplace=True)

>> Calculating confidence-weighted rate for: ['substances'].
   Saving to: substances_confRate. Group Max /Mean / Median / Min: 103 / 4.92 / 2.0 / 1
>> Calculating confidence-weighted rate for: ['titulaires'].
   Saving to: titulaires_confRate. Group Max /Mean / Median / Min: 777 / 26.35 / 5.0 / 1


In [85]:
print("Feature Engineering")
print("\nEncode Variables")


print("Encoding :",categorical)

df['diff']=df['date declar annee']- df['date amm annee']
df.fillna(-1,inplace=True)
# Encoder:
lbl = preprocessing.LabelEncoder()
for col in categorical:
    df[col] = lbl.fit_transform(df[col].astype(str))
    
print("\nText Features")

# Feature Engineering 
df['text_feat'] = df.apply(lambda row: ' '.join([ 
    str(row['titulaires']),
    str(row['substances']), 
    str(row['forme pharma'])]),axis=1) # Group Param Features
    


# Meta Text Features
textfeats = ["text_feat",'libelle', 'titulaires', 'substances', 'forme pharma']

df["text_feat"] = df["text_feat"].apply(lambda x: cleanName(x))
for cols in textfeats:
    df[cols] = df[cols].astype(str) 
    df[cols] = df[cols].astype(str).fillna('missing') # FILL NA
    df[cols] = df[cols].str.lower() # Lowercase all text, so that capitalized words dont get treated differently
    df[cols + '_num_chars'] = df[cols].apply(len) # Count number of Characters
    df[cols + '_num_words'] = df[cols].apply(lambda comment: len(comment.split())) # Count number of Words
    df[cols + '_num_unique_words'] = df[cols].apply(lambda comment: len(set(w for w in comment.split())))
    df[cols + '_words_vs_unique'] = df[cols+'_num_unique_words'] / df[cols+'_num_words'] * 100 # Count Unique Words
df.drop(['libelle', 'titulaires', 'substances', 'forme pharma'],axis=1,inplace=True)
print("\n[TF-IDF] Term Frequency Inverse Document Frequency Stage")
#russian_stop = set(stopwords.words('russian'))

tfidf_para = {
    "analyzer": 'word',
    "token_pattern": r'\w{1,}',
    "sublinear_tf": False,
    "dtype": np.float32,
    "norm": 'l2',
    #"min_df":5,
    #"max_df":.8,
    "smooth_idf":False
}


def get_col(col_name): return lambda x: x[col_name]
##I added to the max_features of the description. It did not change my score much but it may be worth investigating
vectorizer = FeatureUnion([
        ('text_feat',TfidfVectorizer(
            ngram_range=(1, 1),
            #max_features=4000,
            **tfidf_para,
            preprocessor=get_col('text_feat')))])
    
start_vect=time.time()

#Fit my vectorizer on the entire dataset instead of the training rows
#Score improved by .0001
vectorizer.fit(df.to_dict('records'))

ready_df = vectorizer.transform(df.to_dict('records'))
tfvocab = vectorizer.get_feature_names()
print("Vectorization Runtime: %0.2f Minutes"%((time.time() - start_vect)/60))

# Drop Text Cols
textfeats = ["text_feat"]
df.drop(textfeats, axis=1,inplace=True)



Feature Engineering

Encode Variables
Encoding : ['tx rembours', 'voies admin', 'agrement col', 'etat commerc', 'statut', 'statut admin', 'type proc']

Text Features

[TF-IDF] Term Frequency Inverse Document Frequency Stage
Vectorization Runtime: 0.03 Minutes


In [86]:
traindex.shape

(8564,)

In [87]:
testdex.shape

(3671,)

In [88]:
df[:ntrain].shape

(8564, 59)

In [89]:
df[ntrain:].shape

(3671, 59)

In [90]:
########################################################################################################
# Combine Dense Features with Sparse Text Bag of Words Features
X = hstack([csr_matrix(df[:ntrain].values),ready_df[0:traindex.shape[0]]]) # Sparse Matrix
testing = hstack([csr_matrix(df[ntrain:].values),ready_df[traindex.shape[0]:]])
tfvocab = df.columns.tolist() + tfvocab
for shape in [X,testing]:
    print("{} Rows and {} Cols".format(*shape.shape))
print("Feature Names Length: ",len(tfvocab))
#del df
#gc.collect();

print("\nModeling Stage")


X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=23)
    
print("Light Gradient Boosting Regressor")
lgbm_params =  {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'mae',
    'max_depth': 15,
    'num_leaves':320,
    'feature_fraction': 0.9,
    'bagging_fraction': 1,
    # 'bagging_freq': 5,
    'learning_rate': 0.01,
    'verbose': 0
}  


lgtrain = lgb.Dataset(X_train, y_train,
                feature_name=tfvocab,
                categorical_feature = categorical)
lgvalid = lgb.Dataset(X_valid, y_valid,
                feature_name=tfvocab,
                categorical_feature = categorical)

modelstart = time.time()
lgb_clf = lgb.train(
    lgbm_params,
    lgtrain,
    num_boost_round=16000,
    valid_sets=[lgtrain, lgvalid],
    valid_names=['train','valid'],
    early_stopping_rounds=600,
    verbose_eval=500
)

# Feature Importance Plot
f, ax = plt.subplots(figsize=[7,10])
lgb.plot_importance(lgb_clf, max_num_features=50, ax=ax)
plt.title("Light GBM Feature Importance")
plt.savefig('feature_import.png')



8564 Rows and 2032 Cols
3671 Rows and 2032 Cols
Feature Names Length:  2032

Modeling Stage
Light Gradient Boosting Regressor
Train until valid scores didn't improve in 600 rounds.
[500]	train's l1: 0.24602	valid's l1: 0.360056
[1000]	train's l1: 0.206665	valid's l1: 0.342847
[1500]	train's l1: 0.184603	valid's l1: 0.336831
[2000]	train's l1: 0.168877	valid's l1: 0.333677
[2500]	train's l1: 0.157139	valid's l1: 0.331741
[3000]	train's l1: 0.147762	valid's l1: 0.330607
[3500]	train's l1: 0.139257	valid's l1: 0.330026
[4000]	train's l1: 0.132087	valid's l1: 0.329467
[4500]	train's l1: 0.12644	valid's l1: 0.329178
[5000]	train's l1: 0.120969	valid's l1: 0.329031
[5500]	train's l1: 0.116245	valid's l1: 0.328709


KeyboardInterrupt: 

In [ ]:
print("Model Evaluation Stage")
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_valid, lgb_clf.predict(X_valid))))
lgpred = lgb_clf.predict(testing) 

In [ ]:
#Mixing lightgbm with ridge. I haven't really tested if this improves the score or not
#blend = 0.95*lgpred + 0.05*ridge_oof_test[:,0]



In [ ]:
idd=range(1,lgsub.shape[0]+1)
lgsub= pd.DataFrame()
lgsub['id']=idd
lgsub["deal_probability"] = np.exp(lgpred)

In [ ]:
lgsub.head()

In [ ]:
#lgsub['deal_probability'].clip(0.0, 1.0, inplace=True) # Between 0 and 1
lgsub.to_csv("lgsMDF_3.csv",index=False,sep=';')


In [ ]:
0.368-0.392